In [1]:
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import trt_pose.coco
import math
import os
import numpy as np
import traitlets
import pickle 


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-mn8hww0h because the default path (/home/mikyas/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
with open('hand_pose.json', 'r') as f:
    hand_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(hand_pose)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
import torch


WIDTH = 256
HEIGHT = 256
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

if not os.path.exists('resnet18_244x224_epoch_4150_trt.pth'):
    MODEL_WEIGHTS = 'resnet18_244x224_epoch_4150.pth'
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    import torch2trt
    model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    OPTIMIZED_MODEL = 'resnet18_244x224_epoch_4150_trt.pth'
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)


OPTIMIZED_MODEL = 'resnet18_244x224_epoch_4150_trt.pth'
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

In [3]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology,cmap_threshold=0.12, link_threshold=0.15)
draw_objects = DrawObjects(topology)

In [4]:

import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))

In [6]:
from preprocessdata import preprocessdata
preprocessdata = preprocessdata(topology, num_parts)

In [ ]:
from dataloader import dataloader
path = "/home/mikyas/mike_dataset/gestures/hand_dataset/"
label_file = "hand_dataset.json"
test_label = "hand_dataset_test.json"
hand = dataloader(path, label_file, test_label)

In [ ]:
def data_preprocess(images):
    dist_bn_joints_all_data = []
    for im in images:
        im = im[:, ::-1, :]
        data_im = preprocess(im)
        cmap, paf = model_trt(data_im)
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        counts, objects, peaks = parse_objects(cmap, paf)
        joints = preprocessdata.joints_inference(im, counts, objects, peaks)
        dist_bn_joints = preprocessdata.find_distance(joints)
        dist_bn_joints_all_data.append(dist_bn_joints)
    return dist_bn_joints_all_data

In [ ]:
def smaller_dataset(dataset, no_samples_per_class, no_of_classes):
    total_samples_per_class =100
    start = 0
    end = no_samples_per_class
    new_dataset = []
    labels = []
    for i in range(no_of_classes):
        new_data = dataset[start:end]
        start = start+total_samples_per_class
        end = start+no_samples_per_class
        new_dataset.extend(new_data)
        labels.extend([i+1]*no_samples_per_class)
    return new_dataset, labels

In [ ]:
train_images, labels_train = hand.smaller_dataset(hand.train_images,100,6)

In [ ]:
joints_train = data_preprocess(hand.train_images)
joints_test = data_preprocess(hand.test_images)

In [7]:
svm_train = False
if svm_train:
    clf, predicted = preprocessdata.trainsvm(clf, joints_train, joints_test, hand.labels_train, hand.labels_test)
    filename = 'svmmodel_new.sav'
    pickle.dump(clf, open(filename, 'wb'))
else:
    filename = 'svmmodel.sav'
    clf = pickle.load(open(filename, 'rb'))

In [ ]:
preprocessdata.svm_accuracy(clf.predict(joints_test), hand.labels_test)


In [ ]:
clf.predict([joints_test[40],[0]*num_parts*num_parts])

In [ ]:
clf.predict(joints_test)

In [8]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=1)
#camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

In [9]:
import ipywidgets
from IPython.display import display


image_w = ipywidgets.Image(format='jpeg', width=256, height=256)
display(image_w)

Image(value=b'', format='jpeg', height='256', width='256')

In [10]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    joints = preprocessdata.joints_inference(image, counts, objects, peaks)
    dist_bn_joints = preprocessdata.find_distance(joints)
    gesture = clf.predict([dist_bn_joints,[0]*num_parts*num_parts])
    gesture_joints = gesture[0]
    preprocessdata.prev_queue.append(gesture_joints)
    preprocessdata.prev_queue.pop(0)
    preprocessdata.print_label(image, preprocessdata.prev_queue)
    image_w.value = bgr8_to_jpeg(image)


In [11]:
execute({'new': camera.value})

In [12]:
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()

In [ ]:
#camera.running = False